In [1]:
%pip install tqdm
%pip install gensim

Note: you may need to restart the kernel to use updated packages.
  Using cached https://files.pythonhosted.org/packages/70/cf/87b25b265d23498b2b70ce873495cf7ef91394c4baff240210e26f3bc18a/gensim-3.8.3-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3c/f4/41c1d8a69b07b2a087a7e552cbed21111ff36706fec2f1ba9983fba95771/boto3-1.14.20-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/87/a6/1710181d97a6763ccced7f69fff8beea751633af2a101c3d02826cf4acce/botocore-1.17.20-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/07/cb/5f001272b6faeb23c1c9e0acc04d48eaaf5c862c17709d20e3469c6e0139/jmespath-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

from tqdm import tqdm

from arena_util import remove_seen
from arena_util import write_json

from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors
from gensim.parsing.preprocessing import preprocess_string

### 데이터 불러오기

In [2]:
train = pd.read_json('train.json', encoding='UTF-8')
val = pd.read_json('val.json', encoding='UTF-8')
song_meta = pd.read_json('./song_meta.json', encoding='UTF-8')
most_results = pd.read_json('results.json', encoding='UTF-8')
min_count = 10
size = 100
windows = 100
sg = 1

In [3]:
p2v_model = WordEmbeddingsKeyedVectors(size)

In [4]:
# pandas apply 진행상황 보여주기 위한 모듈
tqdm.pandas()

In [9]:
for index, q in tqdm(song_meta.iterrows()):
    try:
        preprocess_string(q['album_name'])
    except:
        print(q['album_name'])




0it [00:00, ?it/s]


1it [00:00,  1.50it/s]


729it [00:00,  2.14it/s]


1409it [00:00,  3.06it/s]


2070it [00:00,  4.37it/s]


2760it [00:01,  6.24it/s]


3478it [00:01,  8.91it/s]


4168it [00:01, 12.73it/s]


4875it [00:01, 18.17it/s]


5623it [00:01, 25.93it/s]


6329it [00:01, 36.98it/s]


7012it [00:01, 52.70it/s]


7692it [00:01, 75.02it/s]


8384it [00:01, 106.68it/s]


9097it [00:01, 151.42it/s]


9782it [00:02, 214.19it/s]


10457it [00:02, 301.38it/s]


11207it [00:02, 423.24it/s]


11948it [00:02, 590.18it/s]


12650it [00:02, 813.78it/s]


13351it [00:02, 1102.27it/s]


14033it [00:02, 1467.15it/s]


14719it [00:02, 1919.78it/s]


15394it [00:02, 2434.29it/s]


16077it [00:03, 3016.49it/s]


16809it [00:03, 3661.67it/s]


17499it [00:03, 4241.79it/s]


18184it [00:03, 4740.83it/s]


18879it [00:03, 5240.14it/s]


19580it [00:03, 5668.53it/s]


20267it [00:03, 5945.89it/s]


20993it [00:03, 6285.59it/s]


21689it [00:03, 6319.00it/s]


22368it [00:03, 6268.83it/s]


230

None





144845it [00:20, 7496.03it/s]


145596it [00:20, 7348.52it/s]


146333it [00:20, 7314.42it/s]


147106it [00:21, 7433.73it/s]


147864it [00:21, 7475.44it/s]


148634it [00:21, 7540.07it/s]


149389it [00:21, 7507.23it/s]


150141it [00:21, 7472.92it/s]


150907it [00:21, 7525.83it/s]


151660it [00:21, 7486.28it/s]


152409it [00:21, 7266.37it/s]


153138it [00:21, 7217.26it/s]


153870it [00:21, 7245.69it/s]


154648it [00:22, 7396.39it/s]


155427it [00:22, 7508.32it/s]


156200it [00:22, 7572.54it/s]


156959it [00:22, 7529.53it/s]


157713it [00:22, 7471.31it/s]


158461it [00:22, 7373.42it/s]


159206it [00:22, 7390.79it/s]


159967it [00:22, 7454.78it/s]


160713it [00:22, 7136.61it/s]


161430it [00:22, 7037.45it/s]


162192it [00:23, 7201.98it/s]


162973it [00:23, 7373.54it/s]


163744it [00:23, 7468.93it/s]


164494it [00:23, 7404.02it/s]


165257it [00:23, 7468.78it/s]


166035it [00:23, 7559.14it/s]


166844it [00:23, 7708.89it/s]


167617it [00:23, 7669.47it/s]


16838

KeyboardInterrupt: 

### Dictionary 만들기

- song_name_dic  
    plylst id와 song name을 분해한 list를 매칭(plylst id - \[song1 분해, song2 분해, ... \])
- total(pandas.Series)  
    song name 분해한 것 + tag + plylst 제목 분해한 것

In [10]:
def preproc_song(song_id):
    song_pre_str = preprocess_string(song_meta.loc[song_id, 'song_name'])
    try:
        song_pre_str += preprocess_string(song_meta.loc[song_id, 'album_name'])
    except:
        pass
    
    return song_pre_str

In [ ]:
song_dic = {} # plylst id - song id
song_name_dic = {} # plylst id - [preprocessed song name]
tag_dic = {} # plylst id - tag
data = pd.concat([train, val])
data = data.set_index('id')

song_dic = data['songs'].to_dict()
tag_dic = data['tags'].to_dict()
data['song_name_token'] = data['songs'].progress_apply(lambda songs : sum([preproc_song(song_id) for song_id in songs], []))
song_name_dic = data['song_name_token'].to_dict()





  0%|          | 0/138086 [00:00<?, ?it/s]



  0%|          | 27/138086 [00:00<08:34, 268.19it/s]



  0%|          | 63/138086 [00:00<07:55, 290.00it/s]



  0%|          | 100/138086 [00:00<07:25, 309.71it/s]



  0%|          | 133/138086 [00:00<07:18, 314.95it/s]



  0%|          | 167/138086 [00:00<07:12, 318.64it/s]



  0%|          | 195/138086 [00:00<07:54, 290.69it/s]



  0%|          | 234/138086 [00:00<07:20, 312.89it/s]



  0%|          | 265/138086 [00:00<08:38, 265.81it/s]



  0%|          | 293/138086 [00:00<08:30, 269.91it/s]



  0%|          | 328/138086 [00:01<07:56, 288.85it/s]



  0%|          | 358/138086 [00:01<08:35, 267.28it/s]



  0%|          | 393/138086 [00:01<07:59, 286.98it/s]



  0%|          | 423/138086 [00:01<08:13, 279.07it/s]



  0%|          | 452/138086 [00:01<08:46, 261.63it/s]



  0%|          | 479/138086 [00:01<09:21, 245.09it/s]



  0%|          | 505/138086 [00:01<09:20, 245.45it/s]



  0%|          | 534/138086 [00:01<08:54

  3%|▎         | 3819/138086 [00:12<08:18, 269.10it/s]



  3%|▎         | 3855/138086 [00:13<07:41, 290.87it/s]



  3%|▎         | 3885/138086 [00:13<08:36, 259.64it/s]



  3%|▎         | 3917/138086 [00:13<08:08, 274.88it/s]



  3%|▎         | 3948/138086 [00:13<07:52, 284.03it/s]



  3%|▎         | 3984/138086 [00:13<07:22, 303.01it/s]



  3%|▎         | 4016/138086 [00:13<07:39, 291.65it/s]



  3%|▎         | 4053/138086 [00:13<07:11, 310.91it/s]



  3%|▎         | 4087/138086 [00:13<07:00, 318.56it/s]



  3%|▎         | 4126/138086 [00:13<06:37, 336.60it/s]



  3%|▎         | 4161/138086 [00:14<06:48, 328.03it/s]



  3%|▎         | 4199/138086 [00:14<06:32, 341.16it/s]



  3%|▎         | 4234/138086 [00:14<06:56, 321.31it/s]



  3%|▎         | 4267/138086 [00:14<07:17, 306.18it/s]



  3%|▎         | 4299/138086 [00:14<07:30, 296.95it/s]



  3%|▎         | 4330/138086 [00:14<07:45, 287.39it/s]



  3%|▎         | 4360/138086 [00:14<07:51, 283.73it/s]



  3%|▎        

  6%|▌         | 7680/138086 [00:25<06:57, 312.09it/s]



  6%|▌         | 7712/138086 [00:26<07:17, 298.24it/s]



  6%|▌         | 7749/138086 [00:26<06:55, 313.93it/s]



  6%|▌         | 7782/138086 [00:26<07:02, 308.09it/s]



  6%|▌         | 7814/138086 [00:26<07:08, 303.75it/s]



  6%|▌         | 7850/138086 [00:26<06:52, 315.43it/s]



  6%|▌         | 7882/138086 [00:26<07:03, 307.31it/s]



  6%|▌         | 7914/138086 [00:26<07:08, 303.49it/s]



  6%|▌         | 7945/138086 [00:26<07:33, 286.87it/s]



  6%|▌         | 7990/138086 [00:26<06:45, 320.90it/s]



  6%|▌         | 8024/138086 [00:26<06:42, 322.94it/s]



  6%|▌         | 8062/138086 [00:27<06:29, 333.92it/s]



  6%|▌         | 8097/138086 [00:27<06:34, 329.66it/s]



  6%|▌         | 8131/138086 [00:27<06:43, 321.96it/s]



  6%|▌         | 8164/138086 [00:27<06:42, 323.13it/s]



  6%|▌         | 8197/138086 [00:27<06:57, 311.08it/s]



  6%|▌         | 8239/138086 [00:27<06:27, 335.40it/s]



  6%|▌        

  9%|▊         | 11807/138086 [00:38<06:58, 301.96it/s]



  9%|▊         | 11854/138086 [00:38<06:17, 334.19it/s]



  9%|▊         | 11889/138086 [00:38<06:19, 332.78it/s]



  9%|▊         | 11924/138086 [00:38<06:34, 319.74it/s]



  9%|▊         | 11957/138086 [00:39<06:46, 310.40it/s]



  9%|▊         | 11989/138086 [00:39<06:53, 304.68it/s]



  9%|▊         | 12031/138086 [00:39<06:25, 327.40it/s]



  9%|▊         | 12065/138086 [00:39<06:22, 329.44it/s]



  9%|▉         | 12099/138086 [00:39<06:30, 322.79it/s]



  9%|▉         | 12141/138086 [00:39<06:05, 344.95it/s]



  9%|▉         | 12178/138086 [00:39<05:57, 351.75it/s]



  9%|▉         | 12214/138086 [00:39<06:02, 347.56it/s]



  9%|▉         | 12250/138086 [00:39<06:19, 331.39it/s]



  9%|▉         | 12286/138086 [00:39<06:13, 336.81it/s]



  9%|▉         | 12322/138086 [00:40<06:12, 337.98it/s]



  9%|▉         | 12357/138086 [00:40<06:58, 300.37it/s]



  9%|▉         | 12388/138086 [00:40<07:04, 295.87it/s]


In [8]:
data = data.reset_index()
total = data.progress_apply(lambda x : song_name_dic[x['id']] + preprocess_string(x['plylst_title']) + tag_dic[x['id']], axis = 1)

100%|██████████| 115071/115071 [00:07<00:00, 15857.17it/s]


### Word2Vec Model Fitting

In [9]:
w2v_model = Word2Vec(total, min_count = min_count, size = size, window = windows, sg = sg)

### Calculate Embedding Vectors

In [10]:
ID = []   
vec = []
embedd = {}
for index, q in tqdm(pd.concat([train, val]).iterrows()):
    tmp_vec = 0
    for song_word in song_name_dic[q['id']]:
        try:
            tmp_vec += w2v_model.wv.get_vector(song_word) / len(song_name_dic[q['id']])
        except KeyError:
            pass
    for tag in tag_dic[q['id']]:
        try:
            tmp_vec += w2v_model.wv.get_vector(tag) / len(tag_dic[q['id']])
        except KeyError:
            pass
    for title_word in preprocess_string(q['plylst_title']):
        try:
            tmp_vec += 2 * w2v_model.wv.get_vector(title_word) / len(preprocess_string(q['plylst_title']))
        except KeyError:
            pass

    if type(tmp_vec) != int:
        embedd[str(q['id'])] = tmp_vec
        ID.append(str(q['id']))  
        vec.append(tmp_vec)

p2v_model.add(ID, vec)

115071it [02:31, 759.14it/s] 


In [24]:
s2v_model = WordEmbeddingsKeyedVectors(size)

ID_song = []
vec_song = []
song_embed = {}
for index, q in tqdm(song_meta.iterrows()):
    tmp_vec = 0
    for word in preprocess_string(q['song_name']):
        try:
            tmp_vec += w2v_model.wv.get_vector(word) / len(preprocess_string( q['song_name']))
        except KeyError:
            pass
    if type(tmp_vec) != int:
        song_embed[q['song_name']] = tmp_vec
        ID_song.append(str(q['id']))
        vec_song.append(tmp_vec)
        
s2v_model.add(ID_song, vec_song)

707989it [03:36, 3269.72it/s]


In [52]:
random_id = 300
random_song_name = song_meta.loc[random_id,'song_name']
similar_songs = [int(x[0]) for x in s2v_model.most_similar(str(random_id), topn = 50)]
print(random_song_name)
song_meta.loc[similar_songs, :]
s2v_model.most_similar(str(random_id), topn = 50)

Vivaldi : Concerto No.4 In F Minor `L`Inverno` RV297 `Winter` - Largo


[('60452', 0.9957778453826904),
 ('381790', 0.9950672388076782),
 ('252871', 0.9922040700912476),
 ('372944', 0.9878601431846619),
 ('351307', 0.9867910742759705),
 ('248161', 0.9867508411407471),
 ('312977', 0.9857215881347656),
 ('170725', 0.9857215881347656),
 ('401079', 0.9857215881347656),
 ('150848', 0.98542320728302),
 ('174269', 0.9849538803100586),
 ('337709', 0.9849538803100586),
 ('1673', 0.9849538803100586),
 ('11392', 0.9849538803100586),
 ('91603', 0.9849538803100586),
 ('706308', 0.9849538803100586),
 ('25197', 0.9849538803100586),
 ('512007', 0.9849538803100586),
 ('688186', 0.9849538803100586),
 ('501341', 0.9849538803100586),
 ('225713', 0.9849538803100586),
 ('37922', 0.9849538803100586),
 ('135514', 0.9849538803100586),
 ('280208', 0.9849538803100586),
 ('279235', 0.9849538803100586),
 ('150173', 0.9849538803100586),
 ('239631', 0.9839617013931274),
 ('132400', 0.9832000136375427),
 ('441159', 0.9832000136375427),
 ('154632', 0.9832000136375427),
 ('443225', 0.98320

In [13]:
answers = []
for index, q in tqdm(val.iterrows()):
    try:
        most_id = [x[0] for x in p2v_model.most_similar(str(q['id']), topn=200)]
        get_song = []
        get_tag = []
        for ID in most_id:
            get_song += song_dic[int(ID)]
            get_tag += tag_dic[int(ID)]
        get_song = list(pd.value_counts(get_song)[:200].index)
        get_tag = list(pd.value_counts(get_tag)[:20].index)
        answers.append({
            "id": q["id"],
            "songs": remove_seen(q["songs"], get_song)[:100],
            "tags": remove_seen(q["tags"], get_tag)[:10],
        })
    except:
        answers.append({
          "id": most_results.loc[index]["id"],
          "songs": most_results.loc[index]['songs'],
          "tags": most_results.loc[index]["tags"],
        }) 

# check and update answer
for n, q in enumerate(answers):
    if len(q['songs'])!=100:
        answers[n]['songs'] += remove_seen(q['songs'], most_results.loc[n]['songs'])[:100-len(q['songs'])]
    if len(q['tags'])!=10:
        answers[n]['tags'] += remove_seen(q['tags'], most_results.loc[n]['tags'])[:10-len(q['tags'])]

23015it [02:43, 141.01it/s]


In [39]:
write_json(answers, "hoho_results.json")

## Class로 만든 버전

In [ ]:
# 목표 : 플레이리스트(노래, 태그, 플레이리스트 제목)이 주어지면, K개의 feature를 뽑도록 한다.
# 자연어 처리 모델을 구축
class PlaylistEmbedding:
    def __init__(self):
        # 파일에서 긁어온다.
        # 긁어올 파일 : train, val, song_meta
        self.train = pd.read_json('./data/train.json', encoding='UTF-8')
        self.val = pd.read_json('./data/val.json', encoding='UTF-8')
        self.song_meta = pd.read_json('./data/song_meta.json', encoding='UTF-8')
        self.most_results = pd.read_json('results.json', encoding='UTF-8')
        self.min_count = 10
        self.size = 100
        self.windows = 100
        self.sg = 1
        self.p2v_model = WordEmbeddingsKeyedVectors(self.size)
    
    #플레이리스트 : (플레이리스트 제목과 모든 송의 제목을 gensim으로 preprocess, 장르도 우겨넣자) 
    def get_dic(self, train, val, song_meta):
        song_dic = {} # plylst id - song id
        song_name_dic = {} # plylst id - [preprocessed song name]
        tag_dic = {} # plylst id - tag
        data = pd.concat([train, val])
        data = data.set_index('id')
        
        song_dic = data['songs'].to_dict()
        tag_dic = data['tags'].to_dict()
        data['song_name_token'] = data['songs'].map(lambda x : sum(list(map(lambda xx : preprocess_string(song_meta.loc[xx, 'song_name']), x)), []))
        song_name_dic = data['song_name_token'].to_dict()
        
        '''
        for index, q in tqdm(data.iterrows()):
            song_name_dic[str(q['id'])] = sum(list(map(lambda x : preprocess_string(song_meta.loc[x]['song_name']), q['songs'])),[])
            song_dic[str(q['id'])] = q['songs']
            tag_dic[str(q['id'])] = q['tags']
        '''
        self.song_dic = song_dic
        self.song_name_dic = song_name_dic
        self.tag_dic = tag_dic
        
        data = data.reset_index()
        total = data.apply(lambda x : song_name_dic[x['id']] + tag_dic[x['id']] + preprocess_string(x['plylst_title']), axis = 1)
        # total = [x for x in total if len(x)>1]
        self.total = total
    
    # word2vec 모델
    def get_w2v(self):
        w2v_model = Word2Vec(self.total, min_count = self.min_count, size = self.size, window = self.windows, sg = self.sg)
        self.w2v_model = w2v_model
    
    # word2vec을 적합하고 embedded vector를 return
    def update_p2v(self, train, val,w2v_model):
        ID = []   
        vec = []
        embedd = {}
        for index, q in tqdm(pd.concat([train, val]).iterrows()):
            tmp_vec = 0
            for song_word in self.song_name_dic[str(q['id'])]:
                try:
                    tmp_vec += w2v_model.wv.get_vector(song_word)
                except KeyError:
                        pass
            for tag in self.tag_dic[str(q['id'])]:
                try:
                    tmp_vec += w2v_model.wv.get_vector(tag)
                except KeyError:
                        pass
            for title_word in preprocess_string(q['plylst_title']):
                #print(q['plylst_title'])
                try:
                    tmp_vec += w2v_model.wv.get_vector(title_word)
                except KeyError:
                        pass
                    
            if type(tmp_vec) != int:
                embedd[str(q['id'])] = tmp_vec
                ID.append(str(q['id']))    
                vec.append(tmp_vec)
                
        self.embedd = embedd
        self.p2v_model.add(ID, vec)
        return embedd
    
    def get_results(self):
        answers = []
        for index, q in tqdm(self.val.iterrows()):
            try:
                most_id = [x[0] for x in self.p2v_model.most_similar(str(q['id']), topn=200)]
                get_song = []
                get_tag = []
                for ID in most_id:
                    get_song += self.song_dic[int(ID)]
                    get_tag += self.tag_dic[int(ID)]
                get_song = list(pd.value_counts(get_song)[:200].index)
                get_tag = list(pd.value_counts(get_tag)[:20].index)
                answers.append({
                    "id": q["id"],
                    "songs": remove_seen(q["songs"], get_song)[:100],
                    "tags": remove_seen(q["tags"], get_tag)[:10],
                })
            except:
                answers.append({
                  "id": self.most_results.loc[index]["id"],
                  "songs": self.most_results.loc[index]['songs'],
                  "tags": self.most_results.loc[index]["tags"],
                }) 
                
        # check and update answer
        for n, q in enumerate(answers):
            if len(q['songs'])!=100:
                answers[n]['songs'] += remove_seen(q['songs'], self.most_results.loc[n]['songs'])[:100-len(q['songs'])]
            if len(q['tags'])!=10:
                answers[n]['tags'] += remove_seen(q['tags'], self.most_results.loc[n]['tags'])[:10-len(q['tags'])]  
        self.answers = answers

In [ ]:
playlist = PlaylistEmbedding()

In [ ]:
playlist.get_dic(playlist.train, playlist.val, playlist.song_meta)

In [ ]:
playlist.get_w2v()

In [ ]:
playlist.update_p2v(playlist.train, playlist.val, playlist.w2v_model)

In [ ]:
playlist.get_results()

In [ ]:
write_json(playlist.answers, "results.json")